In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy[cuda112]

In [ ]:
!python -m spacy download de_core_news_sm

In [ ]:
!python -m spacy download en_core_web_trf

In [ ]:
!pip install -U pip setuptools wheel

In [ ]:
!pip install -U spacy

In [ ]:
!python -m spacy download de_dep_news_trf

In [ ]:
!pip install spacy[transformers,cuda112]

In [ ]:
# Test it

import spacy

spacy.prefer_gpu()

# IMPORT

In [4]:
import spacy
import pickle
import numpy as np

from tqdm import tqdm
from collections import Counter
from datetime import datetime

#spacy.prefer_gpu()
nlp = spacy.load("de_dep_news_trf")
nlp.max_length = 17000000

# Preprocess Text

In [ ]:
from datasets import load_from_disk, load_dataset

common_voice_train = load_dataset("common_voice", "de", split="train", cache_dir="D:\Master\wsl\data")
common_voice_validation = load_dataset("common_voice", "de", split="validation", cache_dir="D:\Master\wsl\data")
common_voice_test = load_dataset("common_voice", "de", split="test", cache_dir="D:\Master\wsl\data")

In [ ]:
common_voice_train = common_voice_train.remove_columns(["path","accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_validation = common_voice_validation.remove_columns(["path","accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["path","accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [ ]:
import re
chars_to_keep = '[^A-Za-zäüöß ]+'

def remove_special_characters_chris(batch):
    batch["sentence"] = re.sub(chars_to_keep, '', batch["sentence"]).lower() + " "
    return batch

In [ ]:
common_voice_train_text = common_voice_train.map(remove_special_characters_chris)
common_voice_validation_text = common_voice_validation.map(remove_special_characters_chris)
common_voice_test_text = common_voice_test.map(remove_special_characters_chris)

In [ ]:
common_voice_train_text.save_to_disk("")
common_voice_validation_text.save_to_disk("")
common_voice_test_text.save_to_disk("")

# Load Text Dataset

In [1]:
from datasets import load_from_disk, load_dataset

train_text_dataset = load_from_disk("E:/Master/data/0_text/train_text")
val_text_dataset = load_from_disk("E:/Master/data/0_text/val_text")
test_text_dataset = load_from_disk("E:/Master/data/0_text/test_text")

In [ ]:
from datasets import load_from_disk, load_dataset

train_text_dataset = load_from_disk("/media/chris/TheFlash/Master/data/0_text/train_text")
val_text_dataset = load_from_disk("/media/chris/TheFlash/Master/data/0_text/val_text")
test_text_dataset = load_from_disk("/media/chris/TheFlash/Master/data/0_text/test_text")

# Process Datasets with spacy

In [ ]:
now = datetime.now()
print("start process train @ ", now)

train_doc_trf = []
for i in tqdm(range(train_text_dataset.shape[0])):
    train_doc_trf.append(nlp(train_text_dataset[i]['sentence'], disable = ['ner', 'parser']))

now = datetime.now()
print("End process train @ ", now)

In [ ]:
with open('train_doc_trf.pickle', 'wb') as handle:
    pickle.dump(train_doc_trf, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
now = datetime.now()
print("start process val @ ", now)

val_doc_trf = []
for i in tqdm(range(val_text_dataset.shape[0])):
    val_doc_trf.append(nlp(val_text_dataset[i]['sentence'], disable = ['ner', 'parser']))

now = datetime.now()
print("End process val @ ", now)

  0%|                                                                                | 4/15588 [00:00<06:41, 38.85it/s]

start process val @  2021-05-29 11:04:33.358617


100%|████████████████████████████████████████████████████████████████████████████| 15588/15588 [07:57<00:00, 32.66it/s]

End process val @  2021-05-29 11:12:30.688812


In [8]:
with open('data_w/val_doc_trf.pickle', 'wb') as handle:
    pickle.dump(val_doc_trf, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
print(len(val_doc_trf))

15588


In [10]:
now = datetime.now()
print("start process test @ ", now)

test_doc_trf = []
for i in tqdm(range(test_text_dataset.shape[0])):
    test_doc_trf.append(nlp(test_text_dataset[i]['sentence'], disable = ['ner', 'parser']))

now = datetime.now()
print("End process test @ ", now)

  0%|                                                                                | 3/15588 [00:00<11:36, 22.39it/s]

start process test @  2021-05-29 11:23:09.630017


100%|████████████████████████████████████████████████████████████████████████████| 15588/15588 [13:52<00:00, 18.72it/s]

End process test @  2021-05-29 11:37:02.469228


In [11]:
with open('data_w/test_doc_trf.pickle', 'wb') as handle:
    pickle.dump(test_doc_trf, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Load processed data

In [ ]:
import pickle

with open('train_doc_trf.pickle', 'rb') as handle:
    train_doc_trf_loaded = pickle.load(handle)
    

with open('val_doc_trf.pickle', 'rb') as handle:
    val_doc_trf_loaded = pickle.load(handle)

with open('test_doc_trf.pickle', 'rb') as handle:
    test_doc_trf_loaded = pickle.load(handle)

## Process Train dataset (text/lemma noun/lemma)

In [ ]:
splits = 8
train_tok_text_splits = {}
for i in range(splits):
    train_tok_text_splits[i] = []
    
splittet_train_trf = np.array_split(train_doc_trf_loaded, splits)

for n, token_list in enumerate(splittet_train_trf):
    for i in tqdm(range(len(token_list))):
        train_tok_text_splits[n] = train_tok_text_splits[n] + [token.text for token in token_list[i] if(not token.is_stop and not token.is_punct)]
        
train_tok_text = train_tok_text_splits[0]

for i in range(1,splits):
    train_tok_text = train_tok_text + train_tok_text_splits[i]

with open('train_tok_text.pickle', 'wb') as handle:
    pickle.dump(train_tok_text, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
splits = 8
train_tok_lemma_splits = {}
for i in range(splits):
    train_tok_lemma_splits[i] = []
    
splittet_train_trf = np.array_split(train_doc_trf_loaded, splits)

for n, token_list in enumerate(splittet_train_trf):
    for i in tqdm(range(len(token_list))):
        train_tok_lemma_splits[n] = train_tok_lemma_splits[n] + [token.lemma_ for token in token_list[i] if(not token.is_stop and not token.is_punct)]
        
train_tok_lemma = train_tok_lemma_splits[0]

for i in range(1,splits):
    train_tok_lemma = train_tok_lemma + train_tok_lemma_splits[i]

with open('train_tok_lemma.pickle', 'wb') as handle:
    pickle.dump(train_tok_lemma, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
splits = 8
train_noun_text_splits = {}
for i in range(splits):
    train_noun_text_splits[i] = []
    
splittet_train_trf = np.array_split(train_doc_trf_loaded, splits)

for n, token_list in enumerate(splittet_train_trf):
    for i in tqdm(range(len(token_list))):
        train_noun_text_splits[n] = train_noun_text_splits[n] + [token.text for token in token_list[i] 
                                                               if(not token.is_stop and not token.is_punct and token.pos_ == "NOUN")]
        
train_nouns_text = train_noun_text_splits[0]

for i in range(1,splits):
    train_nouns_text = train_nouns_text + train_noun_text_splits[i]

with open('train_nouns_text.pickle', 'wb') as handle:
    pickle.dump(train_nouns_text, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
splits = 8
train_noun_lemma_splits = {}
for i in range(splits):
    train_noun_lemma_splits[i] = []
    
splittet_train_trf = np.array_split(train_doc_trf_loaded, splits)

for n, token_list in enumerate(splittet_train_trf):
    for i in tqdm(range(len(token_list))):
        train_noun_lemma_splits[n] = train_noun_lemma_splits[n] + [token.lemma_ for token in token_list[i] 
                                                               if(not token.is_stop and not token.is_punct and token.pos_ == "NOUN")]
        
train_nouns_lemma = train_noun_lemma_splits[0]

for i in range(1,splits):
    train_nouns_lemma = train_nouns_lemma + train_noun_lemma_splits[i]

with open('train_nouns_lemma.pickle', 'wb') as handle:
    pickle.dump(train_nouns_lemma, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Process Datasets with Spacy (old way)

In [ ]:
train_complete_text = ""
val_complete_text = ""
test_complete_text = ""

In [ ]:
print(train_text_dataset.shape[0])
print(val_text_dataset.shape[0])
print(test_text_dataset.shape[0])

## Convert Datasets to String

In [ ]:
from tqdm import tqdm

for i in tqdm(range(train_text_dataset.shape[0])):
    train_complete_text = train_complete_text + train_text_dataset[i]['sentence']

for i in tqdm(range(val_text_dataset.shape[0])):
    val_complete_text = val_complete_text + val_text_dataset[i]['sentence']

for i in tqdm(range(test_text_dataset.shape[0])):
    test_complete_text = test_complete_text + test_text_dataset[i]['sentence']

## Process Datasets with Spacy

In [ ]:
from datetime import datetime

now = datetime.now()
print("start process val @ ", now)

val_doc = nlp(val_complete_text, disable = ['ner', 'parser'])

now = datetime.now()
print("End process val @ ", now)

In [ ]:
now = datetime.now()
print("start process test @ ", now)

test_doc_trf = nlp(test_complete_text, disable = ['ner', 'parser'])

now = datetime.now()
print("End process test @ ", now)

In [ ]:
# all tokens that arent stop words or punctuations
train_all = [token.text
         for token in train_doc
         if not token.is_punct]

# all tokens that arent stop words or punctuations
train_words = [token.text
         for token in train_doc
         if not token.is_stop and not token.is_punct]

# noun tokens that arent stop words or punctuations
train_nouns = [token.text
         for token in train_doc
         if (not token.is_stop and
             not token.is_punct and
             token.pos_ == "NOUN")]

In [ ]:
# noun tokens that arent stop words or punctuations
train_nouns_lemma = [token.lemma_
         for token in train_doc_trf_loaded
         if (not token.is_stop and
             not token.is_punct and
             token.pos_ == "NOUN")]

# Results check WER (old)

In [ ]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

model = Wav2Vec2ForCTC.from_pretrained("./wav2vec2-large-xlsr-ger-chris/checkpoint-51000")

In [ ]:
def map_to_result(batch):
    
    model.to("cuda")
    input_values = processor(
          batch["speech"], 
          sampling_rate=batch["sampling_rate"], 
          return_tensors="pt"
    ).input_values.to("cuda")

    with torch.no_grad():
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]

    return batch

In [ ]:
from datasets import load_metric

wer_metric = load_metric("wer")

In [ ]:
from datasets import load_from_disk, load_dataset

test_sampled = load_from_disk("/media/chris/TheFlash/Master/data/test_sampled")

In [ ]:
results = test_sampled.map(map_to_result)

In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["target_text"])))

In [ ]:
results.save_to_disk("results51_test")

In [ ]:
result_complete_text = ""
for i in tqdm(range(results.shape[0])):
    result_complete_text = result_complete_text + results[i]['pred_str']

In [ ]:
result_doc_trf = nlp(result_complete_text, disable = ['ner', 'parser'])

In [ ]:
result_doc_trf = []
for i in tqdm(range(results.shape[0])):
    result_doc_trf.append(nlp(results[i]['pred_str'], disable = ['ner', 'parser']))

In [ ]:
with open('result_doc_trf.pickle', 'wb') as handle:
    pickle.dump(result_doc_trf, handle, protocol=pickle.HIGHEST_PROTOCOL)